In [1]:
# Only execute once: 
# pip install pennylane_qiskit

## Imports

In [1]:
import pennylane as qml
import pennylane_qiskit as pq
from gate_configurations import circuit

import tensorflow as tf

### -----------------------------------------------------------------------------------------

In [2]:
circ = circuit(4)
circ.build_cascade_rx([0.5, 0.33, 0.67, 1.0])
circ.build_cascade_rz([0.0, 0.0, 1.0, 0.707])
circ.cx_almost_all_neighbors()

print('Param Dict:', circ.get_rotation_params(), '\n')
print(circ.get_quantum_circuit().draw())

Param Dict: {'weights_0': 0.5, 'weights_1': 0.33, 'weights_2': 0.67, 'weights_3': 1.0, 'weights_4': 0.0, 'weights_5': 0.0, 'weights_6': 1.0, 'weights_7': 0.707} 

     ┌─────────┐   ┌───────┐                 
q_0: ┤ Rx(0.5) ├───┤ Rz(0) ├────■────────────
     ├─────────┴┐  ├───────┤  ┌─┴─┐          
q_1: ┤ Rx(0.33) ├──┤ Rz(0) ├──┤ X ├──■───────
     ├──────────┤  ├───────┤  └───┘┌─┴─┐     
q_2: ┤ Rx(0.67) ├──┤ Rz(1) ├───────┤ X ├──■──
     └┬───────┬─┘┌─┴───────┴─┐     └───┘┌─┴─┐
q_3: ─┤ Rx(1) ├──┤ Rz(0.707) ├──────────┤ X ├
      └───────┘  └───────────┘          └───┘
c: 4/════════════════════════════════════════
                                             


In [37]:
weight_shapes = circ.get_rotation_params()

In [38]:
qiskit_circ = circ.get_quantum_circuit()
pennylane_circ = pq.load(qiskit_circ)

circ_drawer = qml.draw(pennylane_circ)
print(circ_drawer())

0: ──RX(0.50)──RZ(0.00)─╭●───────┤  
1: ──RX(0.33)──RZ(0.00)─╰X─╭●────┤  
2: ──RX(0.67)──RZ(1.00)────╰X─╭●─┤  
3: ──RX(1.00)──RZ(0.71)───────╰X─┤  


In [29]:
n_qubits = 4

dev = qml.device('default.qubit', wires=n_qubits, shots=1000)

@qml.qnode(dev)
def circuit(inputs, w0, w1, w2, w3, w4, w5, w6, w7):
    qml.RX(inputs[0], wires=0)
    qml.RX(inputs[1], wires=1)
    qml.RX(inputs[2], wires=2)
    qml.RX(inputs[3], wires=3)
    pennylane_circ()
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(2)), qml.expval(qml.PauliZ(3))

@qml.qnode(dev)
def qnode(inputs, weights_0, weight_1):
    qml.RX(inputs[0], wires=0)
    qml.RX(inputs[1], wires=1)
    qml.Rot(*weights_0, wires=0)
    qml.RY(weight_1, wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))

weight_shapes = {"weights_0": 3, "weight_1": 1}
circuit = qml.QNode(circuit, dev)

In [30]:
circuit

<QNode: wires=4, device='default.qubit', interface='auto', diff_method='best'>

In [31]:
weight_shapes

{'weights_0': 3, 'weight_1': 1}

In [24]:
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=2)

### You must execute the following code to convert a QISKIT circuit into a Pennylane quantum circuit. Look at the example above for more details. 

```
qiskit_circ = qc.get_quantum_circuit()

pennylane_circ = pq.load(qiskit_circ)

# To visualize only
circ_drawer = qml.draw(pennylane_circ)
print(circ_drawer)
```